In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/kaggle/input/titanic/train.csv')
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
extra = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df.drop(['Name','Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df = pd.merge(df, pd.get_dummies(df, columns=['Sex']))
df['Target'] = df['Survived']
df.drop('Survived', axis=1, inplace=True)
df['Age'] = df['Age'].fillna(np.mean(df['Age']))
df.drop('Sex', axis=1,inplace=True)
X = df.iloc[:,:8]
y = df.iloc[:,8]
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
dftest.drop(['Name','Ticket','Cabin','Embarked'], axis=1, inplace=True)
dftest = pd.merge(dftest, pd.get_dummies(dftest, columns=['Sex']))
dftest['Age'] = dftest['Age'].fillna(np.mean(dftest['Age']))
dftest.drop('Sex', axis=1,inplace=True)
dftest['Fare'] = dftest['Fare'].fillna(np.mean(dftest['Fare']))


In [3]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.1, max_depth=3).fit(X_train, y_train)

clf.score(X_test, y_test)

0.8161434977578476

In [6]:
from sklearn.model_selection import GridSearchCV

clf = GradientBoostingClassifier()

grid_values = {'max_depth': [4, 8, 16, 32, 64], 'n_estimators':[16, 32, 64, 100, 200], 'learning_rate':np.logspace(-4,1,6)}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)


GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
                         'max_depth': [4, 8, 16, 32, 64],
                         'n_estimators': [16, 32, 64, 100, 200]})

In [8]:
grid_clf_acc.cv_results_


{'mean_fit_time': array([0.0335196 , 0.06018333, 0.1260777 , 0.19264474, 0.35583334,
        0.07629795, 0.14367456, 0.27576623, 0.47462492, 0.86398478,
        0.12859945, 0.24563489, 0.49703484, 0.79412565, 1.50716372,
        0.13092432, 0.24391837, 0.49494085, 0.77847719, 1.59532189,
        0.12717514, 0.25297461, 0.5231782 , 0.80301638, 1.58147178,
        0.03278251, 0.06518989, 0.11215911, 0.16595044, 0.33066988,
        0.06971884, 0.15057554, 0.29755898, 0.44628959, 0.88725915,
        0.12870564, 0.24535322, 0.48989425, 0.77962184, 1.52626925,
        0.12693248, 0.25809364, 0.50687847, 0.78892355, 1.59437299,
        0.13476706, 0.25793118, 0.50789599, 0.82003345, 1.58394704,
        0.02869296, 0.05594087, 0.10827065, 0.16742554, 0.32940717,
        0.07295303, 0.15376263, 0.31189532, 0.47793345, 0.99291725,
        0.12584686, 0.25588884, 0.50662913, 0.8218555 , 1.7134706 ,
        0.1260046 , 0.25511346, 0.50374665, 0.82995119, 1.59592128,
        0.13288221, 0.25750971,

In [10]:
grid_clf_acc.best_estimator_


GradientBoostingClassifier(max_depth=4, n_estimators=32)

In [11]:
predictions = grid_clf_acc.predict(dftest)

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In [13]:
results = pd.DataFrame({'PassengerId':dftest['PassengerId'], 'Survived':grid_clf_acc.predict(dftest)})

results

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
results.to_csv('submission.csv', index=False)

TypeError: to_csv() got an unexpected keyword argument 'Index'